In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import time
from random import randint

In [2]:
options = webdriver.ChromeOptions() 
options.add_argument("download.default_directory=" + os.getcwd() + "/downloads")

In [3]:
start_page = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="
test_name = "Can Alpaslan" + " " + "CSUN"
name = "Can Alpaslan"
print(start_page)

https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG=


In [4]:
#Use pandas create dataframe (df) object that will store the data
columns = ['name','paper_type','citation','url','abstract','title','collaborators']
citations = pd.DataFrame()
#Place the list of names generated previously in the project into the df object
citations

""


In [5]:
#df1['e'] = Series(np.random.randn(sLength), index=df1.index)
#citations['a'] = pd.Series(test_name, index=citations.index)


Process Flow
-------------------
Search professors from start_page by name
Click Search
Find the first reseach item in the research section
Determine type of publication
Get Abstract if Abstract exists, if not skip to next
Click citation download link
Select option (1 = RIS, 2 = BibTex, 3 = Plain text)
Select option  (1 = citation 2 = citation and abstract)
Download
Go to most recent downloaded file
Read text from that file
Place citation into variable
If abstract exists for citation place into abstract variable if no abstract found insert "No abstract found."

In [14]:
driver = webdriver.Chrome()
driver.implicitly_wait(10) # seconds loading implicitly
google_scholar = "https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG="
#Set driver to go to start_page and search a given name with valid scholar profile
driver.get(google_scholar)
print(driver.current_url)
#Instatiate the webdriver and set to implicitly wait for page objects to load

search_bar = driver.find_element_by_class_name('gs_in_txt')
search_bar.send_keys(test_name)
search_bar.send_keys(Keys.ENTER)


#Set driver to click the first profile given as a search result. Will need to be modified to be more general later.
view_profile_link = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3/a')
view_profile_link.click()

# To get all the links we need to click the Show More multiple times to show the whole links 

gs_bpf_wrapper = driver.find_element_by_id("gsc_bpf_more")

while not gs_bpf_wrapper.get_attribute("disabled"):
    gs_bpf_wrapper.click()
    time.sleep(7)

print("Not in an infinite loop")

#pull the first citation for the page
view_citation_links = driver.find_elements_by_class_name('gsc_a_at')
#print(view_citation_links)

citations = []
patents = []
other = []

citation_file = open('google_scholar_citations.txt', "a")
patent_file = open('google_scholar_patents.txt', "a")

for link in view_citation_links:
    
    title = link.text
    
    citation = {
    "Authors" : "",
    "Title" : "",
    "Date" : "",
    "Publisher": "",
    "Description": "",
    "Journal": "",
    "Issue": "",
    "Volume":"",
    "Pages": ""
    }
    
    patent = {
    "Inventors" : "",
    "Title" : "",
    "Date" : "",
    "Patent Office": "",
    "Description": "",
    "Application Number": "",
    "Patent Number": ""
    }
    
    link.click()

    citation_table = driver.find_element_by_id("gsc_vcd_table")
    citation_table_data_objects= citation_table.find_elements_by_class_name("gs_scl")
    
    data_category = citation_table_data_objects[0].find_element_by_tag_name("div")
    
    if data_category.text == "Authors":
        
        i = 0
        for data in citation_table_data_objects:

            data_category = citation_table_data_objects[i].find_element_by_tag_name("div")
            
            citation_title = driver.find_element_by_id('gsc_vcd_title')
            citation["Title"] = citation_title.text

            if data_category.text == "Authors":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                citation["Authors"] = data_field.text
            elif data_category.text == "Publication date":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                citation["Date"] = data_field.text
            elif data_category.text == "Publisher":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                citation["Publisher"] = data_field.text
            elif data_category.text == "Description":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                citation["Description"] = data_field.text
            elif data_category.text == "Journal":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                citation["Journal"] = data_field.text
            elif data_category.text == "Issue":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                citation["Issue"] = data_field.text
            elif data_category.text == "Volume":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                citation["Volume"] = data_field.text
            elif data_category.text == "Pages":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                citation["Pages"] = data_field.text
            i = i + 1
        citations.append(citation)
        
        citation_record = name + "|" + citation['Title'] + "| " + citation['Authors'] + "| " + citation['Date'] + "| " + citation['Journal'] + "| " + citation['Volume'] + "| " + citation['Issue'] + "| " + citation['Pages'] + "| " + citation['Publisher'] + "| " + citation['Description'] + "\n" 
        citation_file.write(str(citation_record))

                
    elif data_category.text == "Inventors":
        
        for data in citation_table_data_objects:

            i = 0
            
            data_category = citation_table_data_objects[i].find_element_by_tag_name("div")
            
            citation_title = driver.find_element_by_id('gsc_vcd_title')
            citation["Title"] = citation_title.text

            if data_category.text == "Inventors":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                patent["Inventors"] = data_field.text
            elif data_category.text == "Publication date":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                patent["Date"] = data_field.text
            elif data_category.text == "Patent office":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                patent["Patent Office"] = data_field.text
            elif data_category.text == "Description":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                patent["Description"] = data_field.text
            elif data_category.text == "Application number":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                patent["Application Number"] = data_field.text
            elif data_category.text == "Patent number":
                data_field = data.find_element_by_class_name("gsc_vcd_value")
                patent["Patent Number"] = data_field.text
           
            # this is where we would store this record in a pandas dataframe
            i += 1
        patents.append(patent)
        
        patent_record = name + "|" + citation['Title'] + "| " + citation['Inventors'] + "| " + citation['Date'] + "| " + citation['Patent Office'] + "| " + citation['Description'] + "| " + citation['Application Number'] + "| " + citation['Patent Number'] + "\n" 
        patent_file.write(patent_record)
        
       
#     else:
#         other.append("Error: Public ", i , " did not get captured by program; new decision tree required.")
    
    close_table = driver.find_element_by_id("gs_md_cita-d-x")
    
    
    
    #webdriver.ActionChains(driver).move_to_element(close_table ).click(close_table ).perform()
     
    close_table.click()
    time.sleep(3)
    
citation_file.close()
patent_file.close()    

    
#view_citation_link.click()

#Citation information title, publication date, journal, volume, issue, pages, publisher, abstract.

# citations = ['Professor','Authors','Publication_Title', 'URL','Citation']







# citation_table_data = [data.text for data in citation_table_data_objects]
# for data in citation_table_data:
#     if citation_table.find_elements_by_xpath('//div/div')
#     pub_title = driver.find_element_by_class_name('gsc_vcd_title_link')
#     citation["Title"] = pub_title.text


# pub_date = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[2]/div[2]')
# pub_date = pub_date.text
# pub_journal = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[3]/div[2]')
# pub_journal = pub_journal.text
# pub_vol = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[4]/div[2]')
# pub_vol = pub_vol.text
# pub_iss = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[5]/div[2]')
# pub_iss = pub_iss.text
# pub_pgs = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[6]/div[2]')
# pub_pgs = pub_pgs.text
# pub_publisher = driver.find_element_by_xpath('//*[@id="gsc_vcd_table"]/div[7]/div[2]')
# pub_publisher = pub_publisher.text
# pub_abstr = driver.find_element_by_xpath('//*[@id="gsc_vcd_descr"]/div/div')
# pub_abstr = pub_abstr.text 
# print(pub_title)
# print(pub_date)
# print(pub_journal)
# print(pub_vol)
# print(pub_iss)
# print(pub_pgs)
# print(pub_publisher)
# print(pub_abstr)

# Retrieve all the urls of the first page
#research_a_tags = driver.find_elements_by_xpath("html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[2]/div/a")
#urls = [tag.get_attribute("href") for tag in research_a_tags]
#titles = [tag.text for tag in research_a_tags]
#paper_types = driver.find_elements_by_xpath('html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[3]/div/div/span')
#paper_types = [item.text for item in paper_types]
#num_research_items -= 100

#current_url = driver.current_url

#page_count = 2
#while num_research_items > 0:
    #driver.get(current_url + '/' + str(page_count))
    #temp_tags = driver.find_elements_by_xpath("html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[2]/div/a")
    #for tag in temp_tags:
        #urls.append(tag.get_attribute("href"))
        #titles.append(tag.text)
    #temp_papers = driver.find_elements_by_xpath('html/body/div[2]/main/section[2]/div/div[4]/div[2]/div[2]/div/div/div/div/div/div[3]/div/div/span')
    #for paper in temp_papers:
        #paper_types.append(paper.text)
    #num_research_items -= 100
    #page_count += 1

#while "Full-text available" in paper_types:
    #paper_types.remove("Full-text available")
driver.close()

if len(other) > 0:
    for thing in other:
        print(thing)

https://scholar.google.com/citations?hl=en&view_op=search_authors&mauthors=&btnG=
Not in an infinite loop


UnicodeEncodeError: 'charmap' codec can't encode character '\u2010' in position 879: character maps to <undefined>

In [ ]:
print(len(citations) + len(patents))
print(len(citations))
for citation in citations:
    print(citation)

In [84]:
# print(citation['Title'])
# print(citation['Authors'])
# print(citation['Date'])
# print(citation['Journal'])
# print(citation['Volume'])
# print(citation['Issue'])
# print(citation['Pages'])
# print(citation['Publisher'])
# print(citation['Description'])

record = name + "|" + citation['Title'] + "| " + citation['Authors'] + "| " + citation['Date'] + "| " + citation['Journal'] + "| " + citation['Volume'] + "| " + citation['Issue'] + "| " + citation['Pages'] + "| " + citation['Publisher'] + "| " + citation['Description'] + "\n" 


In [86]:
f = open('jury_rigged_scholars.txt', "a")

record = name + "|" + citation['Title'] + "| " + citation['Authors'] + "| " + citation['Date'] + "| " + citation['Journal'] + "| " + citation['Volume'] + "| " + citation['Issue'] + "| " + citation['Pages'] + "| " + citation['Publisher'] + "| " + citation['Description'] + "\n" 
f.write(record)

f.close()

In [42]:
print(len(patents))
for patent in patents:
    print(patent)

1
{'Inventors': 'Automated methods for simulating a biological network\nBE Shapiro, ED Mjolsness, A Levchenko - US Patent 7,319,945, 2008\nCited by 7 Related articles All 2 versions', 'Title': '', 'Date': '', 'Patent Office': '', 'Description': '', 'Application Number': '', 'Patent Number': ''}


In [44]:
# count = 0
# #for url in urls:
# driver = webdriver.Chrome(options=options)
# #time.sleep(randint(5,10))

# driver.get(urls[0])

# # At this point we already have name, title, type, url
# # Need citation, collabs, date, DOI, publisher, edition, series, volume, issue, ISSN, ISBN, handle, book title, 

# if paper_types[0] == 'Article':
#     # this should get the DOI and Abstract in one go
    
#     publication_details = driver.find_elements_by_class_name('publication-details__section')
#     print(publication_details)
    
#     data = publication_details[0].find_elements_by_xpath('//div/div[2]')
    
#     print(data)
    
#     #meta = driver.find_elements_by_xpath('html/body/div[2]/main/section/section/div/div[2]')
#     #print(meta)
#     #doi = meta[0].text.split("")
#     #doi = doi[1]
#     #abstract = meta[1].text
#     # then we'll get the collaborators
    
    
#     # Attempt to get the date
#     #date = driver.find_element_by_xpath('html/body/div[2]/main/section/section/div/div/span[2]')
    
#     #download the citatation
    
#     # take it apart, perhaps improve it
    
#     # build a pandas row
    
#     # add it to the main dataframe
#     # end loop
    
# #print(doi)
# #print(abstract)

    
# #elif paper_type[0] == "Conference Paper"





# driver.close()
#     #count +=1

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

Note that ResearchGate only loads 100 citations per page, if number of citations % 100 = 0 we go to second page